In [1]:
import pandas as pd
import numpy as np
import json
import datetime

In [2]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_level_zips=store_level_zips[['location_id','revenue_flag','zip']].drop_duplicates()
store_level_zips_1601=store_level_zips[store_level_zips['location_id']=="1601"]


In [3]:
open_now_store=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2018-12-22.csv",dtype=str)
open_now_store=open_now_store[open_now_store['week_end_date']=="2018-12-22"]
open_now_store=open_now_store[['location_id','sales']]
open_now_store['sales']=open_now_store['sales'].astype(float)
open_now_store=open_now_store[open_now_store['sales']>0]

store_level_zips=store_level_zips[store_level_zips['location_id'].isin(open_now_store['location_id'].tolist())]
store_level_zips_others_PST=store_level_zips[store_level_zips['location_id']!="1601"].sort_values(['revenue_flag'])
store_level_zips_others_PST=store_level_zips_others_PST.rename(columns={"revenue_flag":"others_high_flag"})
highest_label=store_level_zips_others_PST[['zip','others_high_flag']].drop_duplicates(['zip'])
store_level_zips_others_PS=store_level_zips[(store_level_zips['location_id']!="1601") & (store_level_zips['revenue_flag'].isin(['P','S']))]



In [4]:
store_level_zips_1601=pd.merge(store_level_zips_1601,highest_label,on='zip',how="left")
store_level_zips_1601['others_high_flag']=store_level_zips_1601['others_high_flag'].fillna("T")
store_level_zips_1601_excluded=store_level_zips_1601[(store_level_zips_1601['revenue_flag'].isin(['P','S'])) & (store_level_zips_1601['others_high_flag']=='T')]
store_level_zips_1601_T_and_T=store_level_zips_1601[(store_level_zips_1601['revenue_flag']=="T") & (store_level_zips_1601['others_high_flag']=='T')]

store_level_zips_1601_keep=store_level_zips_1601[store_level_zips_1601['others_high_flag']!="T"]

In [5]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Planner_Request/BL_Zips_For_1601_to_be_excluded_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

In [6]:
store_level_zips_1601_excluded.to_excel(writer,"1_PS_1601_others_T",index=False)
store_level_zips_1601_T_and_T.to_excel(writer,"2_T_1601_others_T",index=False)
store_level_zips_1601_keep.to_excel(writer,"3_others_PS",index=False)
writer.save()